In [1]:
import base58
import binascii
import ecdsa
import hashlib
import json
import os
import subprocess
import time

def dSHA256(data):
    hash_1 = hashlib.sha256(data).digest()
    hash_2 = hashlib.sha256(hash_1).digest()
    return hash_2

def hash160(s):
    '''sha256 followed by ripemd160'''
    return hashlib.new('ripemd160', hashlib.sha256(s).digest()).digest()

def privkey_to_pubkey(privkey):
    signing_key = ecdsa.SigningKey.from_string(privkey, curve=ecdsa.SECP256k1) # Don't forget to specify the curve
    verifying_key = signing_key.get_verifying_key()

    # Use this code block if the address you gave corresponds to the compressed public key
    x_cor = bytes.fromhex(verifying_key.to_string().hex())[:32] # The first 32 bytes are the x coordinate
    y_cor = bytes.fromhex(verifying_key.to_string().hex())[32:] # The last 32 bytes are the y coordinate
    if int.from_bytes(y_cor, byteorder="big", signed=True) % 2 == 0: # We need to turn the y_cor into a number.
        public_key = bytes.fromhex("02" + x_cor.hex())
    else:
        public_key = bytes.fromhex("03" + x_cor.hex())
    return public_key

In [2]:
# Functions related to generating bitcoin addresses

def encode_base58(s):
    BASE58_ALPHABET = '123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz'
    count = 0
    for c in s:  
        if c == 0:
            count += 1
        else:
            break
    num = int.from_bytes(s, 'big')
    prefix = '1' * count
    result = ''
    while num > 0:  
        num, mod = divmod(num, 58)
        result = BASE58_ALPHABET[mod] + result
    return prefix + result

def encode_base58_checksum(b):
    return encode_base58(b + dSHA256(b)[:4])

# https://en.bitcoin.it/wiki/Wallet_import_format
def privkey_to_wif(privkey, compressed_pubkey, testnet):
    if testnet:
        prefix = b"\xEF"
    else:
        prefix = b"\x80"
    
    # if the privkey will correspond to a compressed public key
    if compressed_pubkey: 
        extended = prefix + privkey + b"\x01"
    else:
        extended = prefix + privkey 
        
    extendedchecksum = extended + dSHA256(extended)[:4]
    wif = encode_base58(extendedchecksum)
    
    return wif

# https://learnmeabitcoin.com/guide/wif
def wif_to_privkey(private_key_WIF):
    private_key_full = base58.b58decode(private_key_WIF)
    
    # If the WIF encoding includes the optional "01" byte for compressed privKey,
    # do not include it in the final output.
    if len(private_key_full) == 38:
        private_key = private_key_full[1:-5] 
        print("compressed pubkey")
    else:
        private_key = private_key_full[1:-4] 
        print("not compressed pubkey")
    return private_key


def pk_to_p2pkh(compressed, testnet):
    '''Returns the address string'''
    pk_hash = hash160(compressed)
    if testnet:
        prefix = b'\x6f'
    else:
        prefix = b'\x00'
    return encode_base58_checksum(prefix + pk_hash)

def pk_to_p2sh_p2wpkh(compressed, testnet):
    pk_hash = hash160(compressed)
    redeemScript = bytes.fromhex(f"0014{pk_hash.hex()}")
    rs_hash = hash160(redeemScript)
    if testnet:
        prefix = b"\xc4"
    else:
        prefix = b"\x05"
    return encode_base58_checksum(prefix + rs_hash)

def wif_to_addresses(wif):
#     wif = "cUy9rC6wteKizfu1fgP2abKUWTkJxjqKp2fba91FkU332CFHo6ix"
    privkey = wif_to_privkey(wif)
    public_key = privkey_to_pubkey(privkey)
    p2pkh_address = pk_to_p2pkh(public_key, testnet = True)
    p2sh_p2wpkh_address = pk_to_p2sh_p2wpkh(public_key, testnet = True)

    print("WIF Private key: " + wif)
    print("Private key: " + privkey.hex())
    print("Public key: " + public_key.hex())
    print("Public key hash: " + hash160(public_key).hex())
    print("Address: " + p2pkh_address)
    print("Address: " + p2sh_p2wpkh_address)
    
def privkey_to_addresses(privkey):
    # privkey = bytes.fromhex("AF933A6C602069F1CBC85990DF087714D7E86DF0D0E48398B7D8953E1F03534A")
    public_key = privkey_to_pubkey(privkey)
    p2pkh_address = pk_to_p2pkh(public_key, testnet = True)
    p2sh_p2wpkh_address = pk_to_p2sh_p2wpkh(public_key, testnet = True)

    print("Private key: " + privkey.hex())
    print("Public key: " + public_key.hex())
    print("Public key hash: " + hash160(public_key).hex())
    print("Address: " + p2pkh_address)
    print("Address: " + p2sh_p2wpkh_address)

# Bitcoin core regtest tx tester
This notebook will start up bitcoind in regtest mode, create transactions using python scripts, and broadcast them to validate that they work. Four transactions are tested here:<br><br>

- tx1 initial tx to fund customer. 1 output 
    - Send 11.3 btc to a P2SH-P2WPKH address. This will be used to fund the customer.
- tx2 funding tx with change. 2 outputs
    - 10.2 btc to an escrow (2-of-2 multisig) 
    - 1 btc to a change address
- tx3 merch close. 1 output
    - 11.3 btc to p2wsh with two ways to spend from it:
        - 2 of 2 multisig signed by cust_pk and merch_pk (cust-close-tx)
        - to_self_delay to merch_close_pk (merch-claim-tx)
- tx4 cust close. 2 output
    - 8 btc to p2wsh with two ways to spend from it:
        - to_self_delay to cust_close_pk (cust-claim-tx)
        - using revocation lock to send to merch_close_pk 
    - 2 btc to merch_close_pk (immediately available)
    - 0 btc OP_RETURN 
- tx 5 merch dispute. 1 output
    - 7.9 btc to merch pubkey

Funding private key is used to fund the escrow tx. <br> 
Merch private key and Cust private key are used to generate the pubkeys in the escrow tx. <br>
The Cust private key is also used for the P2WPKH change output.

In [3]:
funding_privkey_hex = "8911111111111111111111111111111111111111111111111111111111111111"
funding_pubkey_hex = privkey_to_pubkey(bytes.fromhex(funding_privkey_hex)).hex()

merch_privkey_hex = "3911111111111111111111111111111111111111111111111111111111111111"
merch_pubkey_hex = privkey_to_pubkey(bytes.fromhex(merch_privkey_hex)).hex()

cust_privkey_hex = "7911111111111111111111111111111111111111111111111111111111111111"
cust_pubkey_hex = privkey_to_pubkey(bytes.fromhex(cust_privkey_hex)).hex()

cust_close_privkey_hex = "7711111111111111111111111111111111111111111111111111111111111111"
cust_close_pubkey_hex = privkey_to_pubkey(bytes.fromhex(cust_close_privkey_hex)).hex()

change_privkey_hex = "6911111111111111111111111111111111111111111111111111111111111111"
change_pubkey_hex = privkey_to_pubkey(bytes.fromhex(change_privkey_hex)).hex()

merch_close_privkey_hex = "3711111111111111111111111111111111111111111111111111111111111111"
merch_close_pubkey_hex = privkey_to_pubkey(bytes.fromhex(merch_close_privkey_hex)).hex()

merch_disp_privkey_hex = "3111111111111111111111111111111111111111111111111111111111111111"
merch_disp_pubkey_hex = privkey_to_pubkey(bytes.fromhex(merch_disp_privkey_hex)).hex()

revocation_secret_hex = "4011111111111111111111111111111111111111111111111111111111111111"
revocation_lock_hex = hash160(bytes.fromhex(revocation_secret_hex)).hex()

Start up regtest mode, delete any history so we are starting from scratch.
Mine 101 blocks

In [4]:
# Make sure bitcoind is not already running
os.system("bitcoin-cli -regtest stop")
time.sleep(1.5) 

# Delete any previous files to restart regtest
os.system("rm -rfv $HOME/Library/Application\ Support/Bitcoin/regtest/")

# start up bitcoind in regtest mode
os.system("bitcoind -regtest -daemon")
time.sleep(1.5)

# generate 101 blocks so we can fund transactions
os.system("bitcoin-cli -regtest generate 101")
blockcount = subprocess.getoutput("bitcoin-cli -regtest getblockcount")

print("blockcount: " + str(blockcount))

blockcount: 101


Generate base58 address for the escrow funder

In [5]:
# Generate p2sh-p2wpkh address to fund the escrow funder
privkey = bytes.fromhex(funding_privkey_hex)
public_key = privkey_to_pubkey(privkey)
p2sh_p2wpkh_address = pk_to_p2sh_p2wpkh(public_key, testnet = True)

# print("Private key: " + privkey.hex())
# print("Public key: " + public_key.hex())
print("Address: " + p2sh_p2wpkh_address)

Address: 2N1jKbzrUkf583t9mM9ePN3rkqdCwryKcf9


Send btc to the escrow funder

In [6]:
txid_1 = subprocess.getoutput("bitcoin-cli -regtest sendtoaddress " + p2sh_p2wpkh_address + " 11.3")
print(txid_1)

1ca81fce918d778942c4389dc79f450d2458eed0c9d8423cba674a31765a55ca


In [7]:
# Find which output index the btc was sent to
raw_tx = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + txid_1)
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + raw_tx)
d = json.loads(decoded)
# print(decoded)

if d["vout"][0]["scriptPubKey"]["addresses"][0] == p2sh_p2wpkh_address:
    index = 0
else:
    index = 1
print("index: " + str(index))

index: 0


In [8]:
os.system("bitcoin-cli -regtest generate 1");

Generate raw escrow funding transaction

In [9]:
raw_escrow_tx = subprocess.getoutput("python funding_tx_with_change.py" 
                        + " --funding_privkey " + funding_privkey_hex
                        + " --txid " + txid_1
                        + " --index " + str(index)
                        + " --amount_btc " + "11.3"
                        + " --merch_pubkey " + merch_pubkey_hex
                        + " --cust_pubkey " + cust_pubkey_hex
                        + " --change_pubkey " + change_pubkey_hex
                        + " --escrow_btc " + "10.2"
                        + " --change_btc " + "1")
print(raw_escrow_tx)

02000000000101ca555a76314a67ba3c42d8c9d0ee58240d459fc79d38c44289778d91ce1fa81c0000000017160014fc5cc2b7bdcb852b225434d133769a551486950affffffff02fff6cb3c00000000220020666c6bfa88ba97c90cb04c7038d56b5854e71a4dd174d79b1260c822a14f791e00e1f50500000000160014f3a8a4335c0ef84806d93315e56965f13d522e5f02473044022072c06d94616e0eb1974ebbf41b55f626639621588118a89ed85804386a62899b02203f61bcce76601ac27afe126991a62f03b686f2000667ab658e7b607a9fed4b29012103e2aa89cce89e9b2d6f09b20a2096226328f114a4ca62e6ea416b4d7c4573086e00000000


Broadcast escrow funding transaction

In [10]:
escrow_txid = subprocess.getoutput("bitcoin-cli -regtest sendrawtransaction " + raw_escrow_tx + " true")
# "true" flag means we are okay with absurdly high tx fee
print(escrow_txid)

5d7f37ce9a753a235dbc7b4bd9684f2280f71fa984b8a3556a956cec5f181200


In [11]:
raw_escrow_tx1 = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + escrow_txid)
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + raw_escrow_tx1)
# print(decoded)

In [12]:
os.system("bitcoin-cli -regtest generate 1");

Create transaction spending from the change output of the escrow transaction

In [13]:
raw_merch_close_tx = subprocess.getoutput("python merch_close.py"        
                + " --cust_privkey " + cust_privkey_hex 
                + " --merch_privkey " + merch_privkey_hex      
                + " --merch_close_pubkey " + merch_close_pubkey_hex   
                + " --to_self_delay " + "05cf" # number of blocks to delay to-self output       
                + " --txid " + escrow_txid
                + " --index " + "0"
                + " --amount_btc " + "10.2"
                + " --output_btc " + "10.1")
print(raw_merch_close_tx)

decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + raw_merch_close_tx)
# print(decoded)

020000000001010012185fec6c956a55a3b884a91ff780224f68d94b7bbc5d233a759ace377f5d0000000000ffffffff018060333c00000000220020c3fae9ae705465ac132b128c84fc011be28c21bff28e165f7cfb776dfbb117ff0400473044022047fd74fe53e6ce70daf2dff16299cb4f9afb98dabe70b763682b333a32402de10220572ae1832a4c8653e9f81d2470d1f880a3487257c7b9b15b039e5b1bf904ba7f01483045022100c494f5a92cc65aa65afc84fbd2adcd500f6ab253cec7eb40709d791d87eaff84022004e9ad1d67f80df7d67b062975f82968309b1f10b601e2dd7bb5593cfd4f2a990147522102f3d17ca1ac6dcf42b0297a71abb87f79dfa2c66278cbb99c1437e6570643ce902103fc43b44cd953c7b92726ebefe482a272538c7e40fdcde5994a62841525afa8d752ae00000000


In [14]:
merch_close_txid = subprocess.getoutput("bitcoin-cli -regtest sendrawtransaction " + raw_merch_close_tx + " true")
print(merch_close_txid)

113ef208b1062829418c2e6f982f11bd4e85dce3235b8d3f5e1acad160ad2b70


In [15]:
merch_close_tx = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + merch_close_txid)
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + merch_close_tx)
print(decoded)

{
  "txid": "113ef208b1062829418c2e6f982f11bd4e85dce3235b8d3f5e1acad160ad2b70",
  "hash": "0a7ea351d5d5a70f8befba36776a6789fcdcac6d9db6ab12248cec043d74a130",
  "version": 2,
  "size": 315,
  "vsize": 150,
  "weight": 597,
  "locktime": 0,
  "vin": [
    {
      "txid": "5d7f37ce9a753a235dbc7b4bd9684f2280f71fa984b8a3556a956cec5f181200",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "",
        "3044022047fd74fe53e6ce70daf2dff16299cb4f9afb98dabe70b763682b333a32402de10220572ae1832a4c8653e9f81d2470d1f880a3487257c7b9b15b039e5b1bf904ba7f01",
        "3045022100c494f5a92cc65aa65afc84fbd2adcd500f6ab253cec7eb40709d791d87eaff84022004e9ad1d67f80df7d67b062975f82968309b1f10b601e2dd7bb5593cfd4f2a9901",
        "522102f3d17ca1ac6dcf42b0297a71abb87f79dfa2c66278cbb99c1437e6570643ce902103fc43b44cd953c7b92726ebefe482a272538c7e40fdcde5994a62841525afa8d752ae"
      ],
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "value"

In [16]:
os.system("bitcoin-cli -regtest generate 1");

# cust-close from merch-close

In [19]:
raw_cust_close_tx = subprocess.getoutput("python cust_close_from_merch_close.py"        
                + " --cust_privkey " + cust_privkey_hex 
                + " --merch_privkey " + merch_privkey_hex      
                + " --merch_close_pubkey " + merch_close_pubkey_hex   
                + " --cust_close_pubkey " + cust_close_pubkey_hex     
                + " --revocation_lock " + revocation_lock_hex      
                + " --merch_disp_pubkey " + merch_disp_pubkey_hex     
                + " --to_self_delay " + "05cf" # number of blocks to delay to-self output       
                + " --txid " + merch_close_txid
                + " --index " + "0"
                + " --amount_btc " + "10.1"
                + " --script_output_btc " + "8"
                + " --merch_output_btc " + "2")
print(raw_cust_close_tx)

# decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + raw_cust_close_tx)
# print(decoded)

02000000000101702bad60d1ca1a5e3f8d5b23e3dc854ebd112f986f2e8c41292806b108f23e110000000000ffffffff030008af2f0000000022002024fdfddacb774716169995d839b3eb07809a5133c4d4a6254615b113f2a4e82e00c2eb0b00000000160014d4354803d10e77eccfc3bf06c152ae694d05d3810000000000000000376a35bf1ad41a96ff238d0b6fee50c8e2cd5bd757ab8803195e272df2310ded35f9958fd0c2847bf73b5b429a716c005d465009bd768641050047304402206ac5eae1d57faed09453e1c99b2a756de93689dd1deea31d79ad08bb11a7976502204771ef4a3b0cc4ee7fd3ed695637f32951a790986a029d0840b84b10b2616e8d01483045022100b41637b87da1c6db0829ac245b442ce9b56988862331a508fdaf60bc83f173d70220443e3d0ded19dcf7106ed044581a8280c017d44f1e0508d69ae6b1bf6b7743970101017263522102f3d17ca1ac6dcf42b0297a71abb87f79dfa2c66278cbb99c1437e6570643ce902103fc43b44cd953c7b92726ebefe482a272538c7e40fdcde5994a62841525afa8d752ae6702cf05b275210342da23a1de903cd7a141a99b5e8051abfcd4d2d1b3c2112bac5c8997d9f12a00ac6800000000
{
  "txid": "92b4cbfda81f0bc21dcbe2c4cb35667148cfdc833f5e5a878b1ce836df20ad44",
  "hash":

In [20]:
cust_close_txid = subprocess.getoutput("bitcoin-cli -regtest sendrawtransaction " + raw_cust_close_tx + " true")
print(cust_close_txid)

92b4cbfda81f0bc21dcbe2c4cb35667148cfdc833f5e5a878b1ce836df20ad44


In [21]:
cust_close_tx = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + cust_close_txid)
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + cust_close_tx)
print(decoded)

{
  "txid": "92b4cbfda81f0bc21dcbe2c4cb35667148cfdc833f5e5a878b1ce836df20ad44",
  "hash": "0fe0171b543d2e47c239ec8a2b8cf5aaa1741646701f0462f46817700295b80c",
  "version": 2,
  "size": 455,
  "vsize": 256,
  "weight": 1022,
  "locktime": 0,
  "vin": [
    {
      "txid": "113ef208b1062829418c2e6f982f11bd4e85dce3235b8d3f5e1acad160ad2b70",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "",
        "304402206ac5eae1d57faed09453e1c99b2a756de93689dd1deea31d79ad08bb11a7976502204771ef4a3b0cc4ee7fd3ed695637f32951a790986a029d0840b84b10b2616e8d01",
        "3045022100b41637b87da1c6db0829ac245b442ce9b56988862331a508fdaf60bc83f173d70220443e3d0ded19dcf7106ed044581a8280c017d44f1e0508d69ae6b1bf6b77439701",
        "01",
        "63522102f3d17ca1ac6dcf42b0297a71abb87f79dfa2c66278cbb99c1437e6570643ce902103fc43b44cd953c7b92726ebefe482a272538c7e40fdcde5994a62841525afa8d752ae6702cf05b275210342da23a1de903cd7a141a99b5e8051abfcd4d2d1b3c2112ba

In [22]:
os.system("bitcoin-cli -regtest generate 1");

In [27]:
raw_merch_dispute_tx = subprocess.getoutput("python merch_dispute.py"   
                + " --merch_disp_privkey " + merch_disp_privkey_hex 
                + " --revocation_secret " + revocation_secret_hex 
                + " --cust_close_pubkey " + cust_close_pubkey_hex   
                + " --output_pubkey " + "024596d7b33733c28101dbc6c85901dffaed0cdac63ab0b2ea141217d1990ad4b1"    
                + " --to_self_delay " + "05cf" # number of blocks to delay to-self output         
                + " --txid " + cust_close_txid
                + " --index " + "0"
                + " --amount_btc " + "8"
                + " --output_btc " + "7.9")
print(raw_merch_dispute_tx)

# decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + raw_merch_dispute_tx)
# print(decoded)

0200000000010144ad20df36e81c8b875a5e3f83dccf48716635cbc4e2cb1dc20b1fa8fdcbb4920000000000ffffffff018071162f00000000160014b7cfef435e3701fdedb7a11164ae44d561698bb904483045022100914931ef9cf744c762e1462601b108bfd3b6b2afadd2e2e3d3b1f64c02c3622402200f2870ab3256e804083300711dbcd050ded4239fa9387bd07d0673e9885050de0120401111111111111111111111111111111111111111111111111111111111111101016463a914bf1ad41a96ff238d0b6fee50c8e2cd5bd757ab8888210253be79afe84fd9342c1f52024379b6da6299ea98844aee23838e8e678a765f7c6702cf05b2752103195e272df2310ded35f9958fd0c2847bf73b5b429a716c005d465009bd76864168ac00000000
{
  "txid": "ee4ed5f09a1d623919b89c3e2978c4ae2819c2f9f6039fb4b5f237f669828a55",
  "hash": "a32341d4f8e10fbb13a44cf0bf69d1a2f4f4608fe95c24ed541b536e34fe02fb",
  "version": 2,
  "size": 294,
  "vsize": 135,
  "weight": 540,
  "locktime": 0,
  "vin": [
    {
      "txid": "92b4cbfda81f0bc21dcbe2c4cb35667148cfdc833f5e5a878b1ce836df20ad44",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": "

In [28]:
merch_dispute_txid = subprocess.getoutput("bitcoin-cli -regtest sendrawtransaction " + raw_merch_dispute_tx + " true")
print(merch_dispute_txid)

ee4ed5f09a1d623919b89c3e2978c4ae2819c2f9f6039fb4b5f237f669828a55


In [29]:
merch_dispute_tx = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + merch_dispute_txid)
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + merch_dispute_tx)
print(decoded)

{
  "txid": "ee4ed5f09a1d623919b89c3e2978c4ae2819c2f9f6039fb4b5f237f669828a55",
  "hash": "a32341d4f8e10fbb13a44cf0bf69d1a2f4f4608fe95c24ed541b536e34fe02fb",
  "version": 2,
  "size": 294,
  "vsize": 135,
  "weight": 540,
  "locktime": 0,
  "vin": [
    {
      "txid": "92b4cbfda81f0bc21dcbe2c4cb35667148cfdc833f5e5a878b1ce836df20ad44",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "3045022100914931ef9cf744c762e1462601b108bfd3b6b2afadd2e2e3d3b1f64c02c3622402200f2870ab3256e804083300711dbcd050ded4239fa9387bd07d0673e9885050de01",
        "4011111111111111111111111111111111111111111111111111111111111111",
        "01",
        "63a914bf1ad41a96ff238d0b6fee50c8e2cd5bd757ab8888210253be79afe84fd9342c1f52024379b6da6299ea98844aee23838e8e678a765f7c6702cf05b2752103195e272df2310ded35f9958fd0c2847bf73b5b429a716c005d465009bd76864168ac"
      ],
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "value": 7.90000000,
    

In [ ]:
os.system("bitcoin-cli -regtest stop")
time.sleep(1.5) 